In [42]:
import os

os.environ['OPENAI_API_KEY']=''
os.environ['LANGCHAIN_TRACING_V2']=''
os.environ['LANGCHAIN_ENDPOINT']=''
os.environ['LANGCHAIN_API_KEY']=''
os.environ['LANGCHAIN_PROJECT']=''

In [43]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [44]:
# 잘못된 형식을 일부러 입력
base_Actor = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

# 잘못된 형식으로 입력된 데이터를 파싱하려고 시도
parser.parse(base_Actor)

OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [45]:
new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(model='gpt-4o'))

In [46]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(model='gpt-4o'))

new_parser

OutputFixingParser(parser=PydanticOutputParser(pydantic_object=<class '__main__.Actor'>), retry_chain=PromptTemplate(input_variables=['completion', 'error', 'instructions'], input_types={}, partial_variables={}, template='Instructions:\n--------------\n{instructions}\n--------------\nCompletion:\n--------------\n{completion}\n--------------\n\nAbove, the Completion did not satisfy the constraints given in the Instructions.\nError:\n--------------\n{error}\n--------------\n\nPlease try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions:')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C2313651C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C2312E2360>, root_client=<openai.OpenAI object at 0x000001C231366450>, root_async_client=<openai.AsyncOpenAI object at 0x000001C231367B00>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrO

In [49]:
final_actor = new_parser.parse(base_Actor)

In [50]:
final_actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])